<a href="https://colab.research.google.com/github/Aandhikads/Aandhikads/blob/main/pcdcomvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import numpy as np

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.
print(x_train.shape[0], '/content/drive/MyDrive/Datasets')
print(x_test.shape[0], '/content/drive/MyDrive/Datasets')

num_classes = 10

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step
60000 /content/drive/MyDrive/Datasets
10000 /content/drive/MyDrive/Datasets


In [4]:
def model_1(x_input):
    x = Dense(512, activation='relu')(x_input)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x_out = Dense(256, activation='relu')(x)
    x_out = Dropout(0.5)(x)
    return x_out

In [5]:
def model_2(x_input):
    x = Dense(800, activation='sigmoid')(x_input)
    x = Dropout(0.5)(x)
    x = Dense(200, activation='sigmoid')(x)
    x = Dropout(0.5)(x)
    x_out = Dense(12)(x)
    return x_out

In [6]:
inputs = Input(shape=(784,))
x      = model_1(inputs)
x_out  = Dense(10, use_bias=False, activation='linear', name='svm')(x)
model = Model(inputs, x_out)

In [7]:
def svm_loss(layer):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)
    
    def categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.maximum(0.0, neg - pos + 1), axis=-1)
        regularization_loss = 0.5*(tf.reduce_sum(tf.square(weights_tf)))
        return regularization_loss + 0.4*hinge_loss
    
    return categorical_hinge_loss

In [8]:
metrics = ['accuracy']
optimizer = tf.keras.optimizers.RMSprop(lr=2e-3, decay=1e-5)
#optimizer = tf.train.AdamOptimizer(1.e-3)

model.compile(optimizer=optimizer, loss=svm_loss(model.get_layer('svm')), metrics=metrics)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [10]:
batch_size = 64
epochs = 20

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
938/938 [==============================] - 13s 14ms/step - loss: 9.9082 - accuracy: 0.9593 - val_loss: 9.9000 - val_accuracy: 0.9666
Epoch 2/20
938/938 [==============================] - 14s 15ms/step - loss: 9.8998 - accuracy: 0.9680 - val_loss: 9.8922 - val_accuracy: 0.9729
Epoch 3/20
938/938 [==============================] - 15s 15ms/step - loss: 9.8953 - accuracy: 0.9719 - val_loss: 9.8929 - val_accuracy: 0.9747
Epoch 4/20
938/938 [==============================] - 14s 15ms/step - loss: 9.8915 - accuracy: 0.9759 - val_loss: 9.8934 - val_accuracy: 0.9737
Epoch 5/20
938/938 [==============================] - 15s 16ms/step - loss: 9.8881 - accuracy: 0.9783 - val_loss: 9.8931 - val_accuracy: 0.9744
Epoch 6/20
938/938 [==============================] - 15s 16ms/step - loss: 9.8869 - accuracy: 0.9795 - val_loss: 9.8902 - val_accuracy: 0.9776
Epoch 7/20
938/938 [==============================] - 16s 17ms/step - loss: 9.8845 - accuracy: 0.9819 - val_loss: 9.8901 - val_accuracy: